Invoking (Java-level) constructors manually can be done in principle, but is annoyingly brittle. Here's a working version.

In [1]:
abstract class Base {
  def f()
}

class FirstDerived(val message: String) extends Base {
  override def f() = {
    println("First: " + message)
  }
}

class SecondDerived(val message: String) extends Base {
  override def f() = {
    println("Second: " + message)
  }
}

def construct_and_use(
    T : Class[_ <: Base] = classOf[FirstDerived],
    m : String = "test") = {
  val ctor = T.getConstructor(getClass, classOf[String])
  val inst : Base = ctor.newInstance(this, m)
  inst.f()
}

construct_and_use()
construct_and_use(classOf[SecondDerived], "Hello world!")

First: test
Second: Hello world!


defined class Base
defined class FirstDerived
defined class SecondDerived
defined function construct_and_use

Neat, right? Now, let's get another handle to a constructor and see what happens.

In [2]:
val T = classOf[FirstDerived]
val ctor2 = T.getConstructor(getClass, classOf[String])

: 

Wait what? Isn't this the same code as above? Well, here's the problem. Jupyter seems to be interpreting Scala classes as *nested Java classes*. This is troublesome because [Java inner-class constructors](https://docs.oracle.com/javase/8/docs/api/java/lang/reflect/Constructor.html#newInstance-java.lang.Object...-) must take their enclosing instance as the first parameter, but different code blocks have incompatible outer classes:

In [3]:
println("required type: " + classOf[FirstDerived].getConstructors()(0).getParameterTypes()(0))
println("current type: " + getClass)

required type: class ammonite.$sess.cmd0$Helper
current type: class ammonite.$sess.cmd2$Helper


What about the CLI then? Well, this code works on `scala main.sc`, but breaks on `scala < main.sc`. In the REPL, classes are *not* nested, so the Java constructors lose their first argument.

Well, after this entire story, you might ask if Scala's own [reflection system](https://docs.scala-lang.org/overviews/reflection/environment-universes-mirrors.html#types-of-mirrors-their-use-cases--examples) is any better...

> A ClassMirror is used for creating invoker mirrors for constructors.

This sounds promising, no?

In [4]:
val ru = scala.reflect.runtime.universe
val m = ru.runtimeMirror(getClass.getClassLoader)

: 

Maybe not on Jupyter then. I can see if I have any better luck with the CLI next time.